In [7]:
import pandas as pd

In [8]:
df = pd.read_pickle("results/test.pkl")

In [60]:
df.columns

Index(['availableLanguages', 'category', 'classification', 'fulltext',
       'identifier', 'keywords', 'lastUpdateDate', 'links', 'publicationDate',
       'referenceDocument', 'relatedProject', 'title', 'typesOfAction',
       'links2', 'entities', 'title_mentions', 'identifier_mentions', 'cites',
       'targets'],
      dtype='object')

In [63]:
df['links'] = df['links'].map(lambda x: [l for l in x if "@" not in l])

In [84]:
output = df[['title','links','targets']]

,title,links,targets
0,h2020-infoday-eosc-call-topics_en,"[http://observatory.euroris-, net.eu/euroris/,...","[observatory, , , d4science, user-driven e-inf..."
1,Guide to the submission and evaluation process,"[http://eur, http://eur]","[eur, eur, classified information, part b]"
2,part H of the General Annexes of the General W...,[],[]
3,H2020 Work Programme 2016-17: Research infrast...,"[http://s3platform.jrc.ec.europa.eu/eye, http:...","[eye, in_your_country_en, index_en, financial ..."
4,2013_arctic_research_plan,"[http://www.whitehouse.gov/ostp/nstc, http://w...","[nstc, ostp, 2009, cgi, www, circ1370, objecti..."
5,EOSC Infoday - European Open Science Cloud for...,[],[user-driven e-infrastructure innovation]
6,Financial support to Third Parties,[],[financial support to third parties]
7,Annotated Grant Agreement,[http://ec.europa.eu/research/participants/dat...,"[unit-, unit-, unit, unit%20costs_clinical_stu..."
8,Legal basis: Horizon 2020 Rules for Participation,[],"[financial support to third parties, classifie..."
9,India,[http://eeas.europa.eu/delegations/india/eu_in...,"[index_en, RI@eeas.europa, india]"


In [91]:
output[['title', 'target_links']].to_csv("results/target_links.csv")

In [10]:
from gensim import corpora
from gensim import corpora, models, similarities
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

/home/chris/anaconda3/envs/masso/lib/python3.5/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [11]:
stop_en = set(stopwords.words('english'))

In [37]:
fulltexts = df['fulltext']
fulltexts = [" ".join(ft) for ft in fulltexts]

In [38]:
fulltexts = [ft.replace('\n  \n', '\n') for ft in fulltexts]
fulltexts = [ft.replace('\n \n', '\n') for ft in fulltexts]
fulltexts = [ft.replace('\n\n', '\n') for ft in fulltexts]
fulltexts = [ft.replace('\n', ' ') for ft in fulltexts]

In [41]:
paragraphs = [ft.split('\x0c') for ft in fulltexts]

In [43]:
texts = [[word for word in document.lower().split() if word not in stop_en] for document in fulltexts]

In [44]:
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
texts = [[token for token in text if frequency[token] > 1] for text in texts]

In [45]:
dictionary = corpora.Dictionary(texts)

In [46]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [47]:
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

In [48]:
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=10)
corpus_lsi = lsi[corpus_tfidf]
lsi.show_topics(num_words=5, formatted=False)

[(0,
  [('\uf0b7', 0.15499163593052784),
   ('horizon', 0.12364833665296539),
   ('2020', 0.11851644866757204),
   ('data', 0.11311679865192817),
   ('shall', 0.11100452456603446)]),
 (1,
  [('macumba', -0.58012538714939976),
   ('marine', -0.23005170078920023),
   ('marieke', -0.12938596582699208),
   ('isolation', -0.12279423020290656),
   ('industry', -0.12103755681018817)]),
 (2,
  [('\uf0b7', 0.28937752218354695),
   ('data', -0.15216380596805451),
   ('macumba', 0.14003043789898384),
   ('—', 0.1282053303780995),
   ('\uf02d', 0.1256416930744024)]),
 (3,
  [('entities', -0.16993955726759163),
   ('legal', -0.15094699561413419),
   ('helpdesk', 0.1496882429072342),
   ('shall', -0.1488322078907858),
   ('•', 0.14535077112950198)]),
 (4,
  [('eloise', -0.26143303224071851),
   ('coastal', -0.26053459962996028),
   ('zone', -0.2472875724809766),
   ('\uf0b7', -0.22711624889427548),
   ('meeting', -0.18936416118327729)]),
 (5,
  [('e', 0.34419135780974042),
   ('n', 0.300874058700600

In [269]:
lda = models.LdaModel(corpus_tfidf, num_topics=10, id2word=dictionary)
corpus_lda = lda[corpus_tfidf]
lda.show_topics(num_words=5, formatted=False)

[(0,
  [('tdm', 0.00054387045504953354),
   ('eloise', 0.00046918628147115494),
   ('seafood', 0.00046769456503912405),
   ('financial', 0.00036635679424774123),
   ('ecsafeseafood', 0.00036251702104758877)]),
 (1,
  [('cloud', 0.00058084221046827305),
   ('macumba', 0.00049395620147073098),
   ('chinese', 0.00044152066484748006),
   ('hnscicloud', 0.00040500835213018904),
   ('4.10.2016', 0.00036737981110933387)]),
 (2,
  [('elixir', 0.00053020854993893196),
   ('bon', 0.00044771076902634036),
   ('zone', 0.00036010182149326169),
   ('ghg-europe', 0.00035826384089057402),
   ('\uf02d', 0.00034369501572635347)]),
 (3,
  [('rri', 0.00061282273825992363),
   ('macumba', 0.00045764713250801117),
   ('microbiome', 0.00040074708401738671),
   ('toolkit', 0.00038125011767269312),
   ('proposals', 0.00038014064893169923)]),
 (4,
  [('costs', 0.00070883521849320747),
   ('\uf0b7', 0.00053075461552708431),
   ('trl', 0.00049187437958808883),
   ('\uf0a7', 0.00045661161064979478),
   ('training'

In [50]:
from itertools import chain

In [51]:
sentences = list(chain.from_iterable([sent_tokenize(ft) for ft in fulltexts]))
sentences = [word_tokenize(s) for s in sentences]

In [52]:
bigram_transformer = models.Phrases(sentences, min_count=4)

In [53]:
w2vModel = models.Word2Vec(bigram_transformer[sentences], size=100, window=5, min_count=3, workers=4, iter=50, sg=1)

/home/chris/anaconda3/envs/masso/lib/python3.5/site-packages/gensim/models/phrases.py:248: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [59]:
w2vModel.most_similar_cosmul(topn=20, positive=['open','science'])

[('all_fields', 0.5907211303710938),
 ('societal_actors', 0.5606890320777893),
 ('dialogue', 0.5606517195701599),
 ('data-driven', 0.5515134930610657),
 ('technology', 0.5489401817321777),
 ('frontier', 0.5349248647689819),
 ('thinking', 0.5319522619247437),
 ('new_talent', 0.5312091708183289),
 ('employability', 0.5277576446533203),
 ('nationally', 0.5257960557937622),
 ('visionary', 0.5249704122543335),
 ('encouraged', 0.5246412754058838),
 ('computing_intensive', 0.5239355564117432),
 ('between_academia', 0.5228273272514343),
 ('foster', 0.5209313035011292),
 ('circulate', 0.5207675695419312),
 ('knowledge', 0.5184003710746765),
 ('advancement', 0.5181301236152649),
 ('participatory', 0.5174267292022705),
 ('labour_market', 0.517035186290741)]

In [57]:
w2vModel.n_similarity(['open','science'], ['open','research'])

0.85039785054331818